In [1]:
import pandas
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
import xgboost as xgb

In [ ]:
dtrain = xgb.DMatrix('trainset.svm.txt')
dtest = xgb.DMatrix('testset.svm.txt')

In [ ]:
param = {'bst:max_depth':4, 'bst:eta':1, 'objective':'binary:logistic' }
param['nthread'] = 2
param['eval_metric'] = 'auc'

In [ ]:
evallist  = [(dtest,'eval'), (dtrain,'train')]

In [ ]:
num_round = 
bst = xgb.train( param, dtrain, num_round, evallist )